In [5]:
import streamlit as st
import pandas as pd
import geopandas as gpd
from pyproj import CRS
from io import BytesIO
import folium
from streamlit_folium import st_folium

# Diccionario de sistemas de referencia
crs_dict = {
    "Latitud y longitud (EPSG:4326)": 4326,
    "Posgar 94, Faja 2 (EPSG:22182)": 22182
}

# 1. Cargar archivo Excel
st.title("Aplicación de Transformación de Coordenadas")
uploaded_file = st.file_uploader("Sube un archivo Excel", type="xlsx")

if uploaded_file:
    df = pd.read_excel(uploaded_file, decimal=',')
    st.write("Datos cargados:")
    st.write(df.head())

    # 2. Selección de columnas para X y Y
    x_col = st.selectbox("Selecciona la columna para X", df.columns)
    y_col = st.selectbox("Selecciona la columna para Y", df.columns)

    # 3. Sistema de referencia original
    crs_origen_name = st.selectbox("Sistema de referencia de origen", list(crs_dict.keys()))
    crs_origen = crs_dict[crs_origen_name]

    # 4. Sistema de referencia de destino
    crs_destino_name = st.selectbox("Sistema de referencia de destino", list(crs_dict.keys()))
    crs_destino = crs_dict[crs_destino_name]

    # Transformación y reproyección
    if st.button("Transformar y re-proyectar"):
        # Crear el GeoDataFrame a partir de los datos cargados
        gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[x_col], df[y_col]), crs=CRS.from_epsg(crs_origen))

        # 5. Reproyectar al CRS de destino
        gdf = gdf.to_crs(epsg=crs_destino)

        # 6. Extraer nuevas coordenadas y añadirlas al DataFrame original
        df['X_nuevo'] = gdf.geometry.x
        df['Y_nuevo'] = gdf.geometry.y

        # 7. Mostrar el GeoDataFrame re-proyectado
        st.write("Datos transformados:")
        st.write(df.head())

        # 8. Visualizar en mapa interactivo
        st.subheader("Mapa interactivo de los puntos transformados")
        centro_mapa = [gdf.geometry.y.mean(), gdf.geometry.x.mean()]
        mapa = folium.Map(location=centro_mapa, zoom_start=5)
        for _, row in gdf.iterrows():
            folium.Marker(location=[row.geometry.y, row.geometry.x]).add_to(mapa)
        st_folium(mapa, width=700, height=500)

        # 9. Descargar el DataFrame con las nuevas coordenadas
        output = BytesIO()
        df.to_excel(output, index=False)
        output.seek(0)
        st.download_button(
            label="Descargar Excel con nuevas coordenadas",
            data=output,
            file_name="datos_transformados.xlsx",
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )
